In [1]:
import numpy as np
import tensorflow as tf
import sys
import time
from datetime import timedelta
import tensorflow.contrib.keras as kr
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.utils import shuffle

import moxing as mox
mox.file.shift('os', 'mox')

INFO:root:Using MoXing-v1.14.1-ddfd6c9a
INFO:root:Using OBS-Python-SDK-3.1.2


In [2]:
trainDataPath = "s3://corpus-2/dataset/corpus_5_for_train.txt"
testDataPath = "s3://corpus-2/dataset/corpus_5_new.txt"
vocabPath = "s3://corpus-text-classification1/data/glove.6B.100d.txt"

In [3]:
split_info = {
    "random": False,
    "expert": [20, 4],
    "bundle": [920, 1],
    "table": [37, 3]
}


def dataset_split(info):
    if info:
        [num, pi] = info
        train_data = [[] for i in range(num)]
        test_data = [[] for i in range(num)]
        
        with open(trainDataPath, "r", encoding='utf-8') as fp:
            for line in fp.readlines():
                word = line.split()
                info = word[0].split(":")
                index = int(info[pi]) - 1
                label = int(info[0])
                content = word[1:]
                train_data[index].append([content,label])
                
        with open(testDataPath, "r", encoding='utf-8') as fp:
            for line in fp.readlines():
                word = line.split()
                info = word[0].split(":")
                index = int(info[pi]) - 1
                label = int(info[0])
                content = word[1:]
                test_data[index].append([content,label])

        for i in range(num):
            # np.random.shuffle(train_data[i])
            train_data[i], test_data[i] = shuffle(train_data[i], test_data[i])
            train_data[i] = np.asarray(train_data[i])
            test_data[i] = np.asarray(test_data[i])

        train_data, test_data = shuffle(train_data, test_data)
        return train_data, test_data
    
    
    train_data = []
    test_data = []
    
    with open(trainDataPath, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            word = line.split()
            label = int(word[0].split(":")[0])
            content = word[1:]
            train_data.append([content,label])
            
    with open(testDataPath, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            word = line.split()
            label = int(word[0].split(":")[0])
            content = word[1:]
            test_data.append([content,label])
            
    train_data, test_data = shuffle(train_data, test_data)
    return np.asarray(train_data), np.asarray(test_data)


def mergeData(data_x, data_y):
    merge_x = data_x[0]
    merge_y = data_y[0]
    for i in range(1,len(data_x)):
        merge_x = np.r_[merge_x,data_x[i]]
        merge_y = np.r_[merge_y,data_y[i]]
        
    return merge_x, merge_y


def train_split_data(model, train_data, test_data, split_type):
    
    print(split_type)
    
    test_acc = []
    fold_id = 0
    
    if split_type != "random":
        tx = []
        ty = []
        for ti in train_data:
            x_train, y_train = process_file(ti[:,0], ti[:,1], word_to_id, num_classes, seq_length)
            tx.append(x_train)
            ty.append(y_train)

        tx = np.asarray(tx)
        ty = np.asarray(ty)

        te_x = []
        te_y = []
        for ti in test_data:
            x_test, y_test = process_file(ti[:,0], ti[:,1], word_to_id, num_classes, seq_length)
            te_x.append(x_test)
            te_y.append(y_test)

        te_x = np.asarray(te_x)
        te_y = np.asarray(te_y)
        
        for train_i, test_i in kf.split(tx):
            fold_id += 1
            print("Fold: ", fold_id)
            train_x, train_y = mergeData(tx[train_i],ty[train_i])
            test_x, test_y = mergeData(te_x[test_i],te_y[test_i])
            test_acc.append(model_train(model, train_x, train_y, test_x, test_y, categories))
        
    else:
        tx, ty = process_file(train_data[:,0], train_data[:,1], word_to_id, num_classes, seq_length)
        te_x, te_y = process_file(test_data[:,0], test_data[:,1], word_to_id, num_classes, seq_length)
        # print(len(tx),len(tx[0]),len(tx[1]))

        for train_i, test_i in kf.split(tx):
            fold_id += 1
            print("Fold: ", fold_id)
            test_acc.append(model_train(model,tx[train_i], ty[train_i],te_x[test_i], te_y[test_i], categories))
        
    print(test_acc)
    print("%s, %s, %s, %s" % (np.mean(test_acc),np.std(test_acc),np.std(test_acc,ddof=1),np.var(test_acc)))
    return test_acc

In [4]:
def loadGloVe(filename):
    vocab = []
    embd = []
    print('Loading GloVe!')
    # vocab.append('unk') #装载不认识的词
    # embd.append([0] * emb_size) #这个emb_size可能需要指定
    file = open(filename,'r',encoding='utf-8')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append([float(ei) for ei in row[1:]])
    file.close()
    print('Completed!')
    return vocab,embd


def process_file(contents, labels, word_to_id, num_classes, pad_max_length):
    """
    将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
    """
    # contents, labels = readfile(filePath)
    data_id, label_id = [], []
    # 将文本内容转换为对应的id形式
    for i in range(len(contents)):
        data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
        label_id.append(labels[i] - 1)  # label_id.append(cat_to_id[labels[i]])
    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, pad_max_length)
    ''' https://blog.csdn.net/TH_NUM/article/details/80904900
    pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
        sequences：浮点数或整数构成的两层嵌套列表
        maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
        dtype：返回的numpy array的数据类型
        padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补
        truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
        value：浮点数，此值将在填充时代替默认的填充值0
    '''
    y_pad = kr.utils.to_categorical(label_id, num_classes=num_classes)  # 将标签转换为one-hot表示
    ''' https://blog.csdn.net/nima1994/article/details/82468965
    to_categorical(y, num_classes=None, dtype='float32')
        将整型标签转为onehot。y为int数组，num_classes为标签类别总数，大于max(y)（标签从0开始的）。
        返回：如果num_classes=None，返回len(y) * [max(y)+1]（维度，m*n表示m行n列矩阵，下同），否则为len(y) * num_classes。
    '''
    return x_pad, y_pad

In [5]:
categories = ['Retrieve Value', 'Filter', 'Compute Derived Value', 'Find Extremum', 'Sort', 
                  'Determine Range', 'Characterize Distribution', 'Find Anomalies', 'Cluster', 'Correlate']
num_classes = len(categories)

vocab, embd = loadGloVe(vocabPath)
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)
word_to_id = dict(zip(vocab, range(vocab_size)))

print(len(embedding),embedding_dim,vocab_size)
 
seq_length = 41  # seq_length = 37  TREC

Loading GloVe!
Completed!
400000 100 400000


In [9]:
def batch_iter(x_pad, y_pad, batch_size):
    """生成批次数据"""
    data_len = len(x_pad)
    num_batch = int((data_len - 1) / batch_size) + 1
    # np.arange()生成0到data_len的等差数列，默认等差为1；np.random.permutation()打乱生成的等差序列的顺序
    # 下面三句语句是为了将训练或测试文本的顺序打乱，因为原文本中每个分类的样本全部挨在一起，这样每个batch训练的都是同一个分类，不太好，打乱后每个batch可包含不同分类
    indices = np.random.permutation(np.arange(data_len))
    x_shuffle = x_pad[indices]
    y_shuffle = y_pad[indices]

    # 返回所有batch的数据
    for i in range(num_batch):
        start_id = i * batch_size
        end_id = min((i + 1) * batch_size, data_len)
        yield x_shuffle[start_id:end_id], y_shuffle[start_id:end_id]
        
        
def evaluate(sess, model, x_pad, y_pad, loss1, acc1, batch_size):
    """评估在某一数据上的准确率和损失"""
    data_len = len(x_pad)
    batch_eval = batch_iter(x_pad, y_pad, batch_size)  # 128
    total_loss = 0.0
    total_acc = 0.0
    for x_batch1, y_batch1 in batch_eval:
        batch_len = len(x_batch1)
        feed_dict1 = {model.inputX: x_batch1, model.inputY: y_batch1, model.dropoutKeepProb: 1.0}
        lossTmp, accTmp = sess.run([loss1, acc1], feed_dict=feed_dict1)
        total_loss += lossTmp * batch_len
        total_acc += accTmp * batch_len

    return total_loss / data_len, total_acc / data_len


def model_train(model, x_train, y_train, x_val, y_val, categories):
    
    # save_path = "%s/%s/%s/%s" % (savePath, split_type, fold_id, fold_id)
    # 创建session
    session = tf.Session()
    session.run(tf.global_variables_initializer())

    print('Training and evaluating...')
    
    total_batch = 0  # 总批次
    best_acc_train = 0.0  # 最佳验证集准确率
    last_improved = 0  # 记录上一次提升批次
    require_improvement = 400  # 如果超过1000轮未提升，提前结束训练
    print_per_batch = 100
    flag = False

    for epoch in range(num_epochs):  # 20
        start_time = time.time()
        
        print('Epoch:', epoch + 1)
        batch_train = batch_iter(x_train, y_train, batch_size)
        for x_batch, y_batch in batch_train:
            feed_dict = {model.inputX: x_batch, model.inputY: y_batch, model.dropoutKeepProb: dropout_keep_prob}
            session.run(model.trainOp, feed_dict=feed_dict)  # 运行优化
            total_batch += 1

            if total_batch % print_per_batch == 0:
                # 每多少轮次输出在训练集和验证集上的性能
                feed_dict[model.dropoutKeepProb] = 1.0
                loss_train, acc_train = session.run([model.loss, model.acc], feed_dict=feed_dict)
                loss_val, acc_val = evaluate(session, model, x_val, y_val, model.loss, model.acc, 64)
                if acc_val > best_acc_train:
                    # 保存最好结果
                    best_acc_train = acc_val
                    last_improved = total_batch
                    # saver.save(sess=session, save_path=save_path)
                    improved_str = '*'
                else:
                    improved_str = ''
                
                duration = time.time() - start_time
                output = 'Iter: {:>1}, Train Loss: {:>6.4}, Train Acc: {:>6.2%}, Val Loss: {:>6.4}, Val Acc: {:>6.2%}, Time: {:.2f}s {}'
                print(output.format(total_batch, loss_train, acc_train, loss_val, acc_val, duration, improved_str))

            if total_batch - last_improved > require_improvement:
                # 验证集正确率长期不提升，提前结束训练
                print("No optimization for a long time, auto-stopping...")
                
                test_data_len = len(x_val)
                test_num_batch = int((test_data_len - 1) / batch_size) + 1

                y_test_cls = np.argmax(y_val, 1)  # 获得类别
                y_test_pred_cls = np.zeros(shape=len(x_val), dtype=np.int32)  # 保存预测结果  len(x_test) 表示有多少个文本

                for i in range(test_num_batch):  # 逐批次处理
                    start_id = i * batch_size
                    end_id = min((i + 1) * batch_size, test_data_len)
                    feed_dict = {
                        model.inputX: x_val[start_id:end_id],
                        model.dropoutKeepProb: 1.0
                    }
                    y_test_pred_cls[start_id:end_id] = session.run(model.y_pred_cls, feed_dict=feed_dict)

                accuracy_score = metrics.accuracy_score(y_test_cls, y_test_pred_cls)
                # 评估
                print("Precision, Recall and F1-Score...")
                print(metrics.classification_report(y_test_cls, y_test_pred_cls, target_names=categories))
                '''
                sklearn中的classification_report函数用于显示主要分类指标的文本报告．在报告中显示每个类的精确度，召回率，F1值等信息。
                    y_true：1维数组，或标签指示器数组/稀疏矩阵，目标值。 
                    y_pred：1维数组，或标签指示器数组/稀疏矩阵，分类器返回的估计值。 
                    labels：array，shape = [n_labels]，报表中包含的标签索引的可选列表。 
                    target_names：字符串列表，与标签匹配的可选显示名称（相同顺序）。 
                    原文链接：https://blog.csdn.net/akadiao/article/details/78788864
                '''

                # 混淆矩阵
                print("Confusion Matrix...")
                cm = metrics.confusion_matrix(y_test_cls, y_test_pred_cls)
                '''
                混淆矩阵是机器学习中总结分类模型预测结果的情形分析表，以矩阵形式将数据集中的记录按照真实的类别与分类模型作出的分类判断两个标准进行汇总。
                这个名字来源于它可以非常容易的表明多个类别是否有混淆（也就是一个class被预测成另一个class）
                https://blog.csdn.net/u011734144/article/details/80277225
                '''
                print(cm)
                
                flag = True
                break  # 跳出循环
        if flag:  # 同上
            break

    session.close()
    return accuracy_score

In [7]:
# 构建adversarailLSTM模型
class AdversarailLSTM(object):

    def __init__(self, wordEmbedding):
        # 定义输入
        self.inputX = tf.placeholder(tf.int32, [None, seq_length], name='inputX')
        self.inputY = tf.placeholder(tf.int32, [None, num_classes], name='inputY')

        self.dropoutKeepProb = tf.placeholder(tf.float64, name='keep_prob')

        # 词嵌入层
        with tf.name_scope("wordEmbedding"):
            wordEmbedding = tf.Variable(initial_value=wordEmbedding)
            self.embeddedWords = tf.nn.embedding_lookup(wordEmbedding, self.inputX)

        # 计算softmax交叉熵损失
        with tf.name_scope("loss"):
            with tf.variable_scope("Bi-LSTM", reuse=None):
                self.predictions = self._Bi_LSTMAttention(self.embeddedWords)
                # self.y_pred_cls = tf.cast(tf.greater_equal(self.predictions, 0.5), tf.float32, name="binaryPreds")
                self.y_pred_cls = tf.argmax(tf.nn.softmax(self.predictions),1)  # 预测类别 tf.argmax：返回每一行或每一列的最大值 1为里面（每一行），0为外面（每一列）
                # losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
                losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
                loss = tf.reduce_mean(losses)

        
        with tf.name_scope("perturloss"):
            with tf.variable_scope("Bi-LSTM", reuse=True):
                perturWordEmbedding = self._addPerturbation(self.embeddedWords, loss)
                print("perturbSize:{}".format(perturWordEmbedding))
                perturPredictions = self._Bi_LSTMAttention(perturWordEmbedding)
                # perturLosses = tf.nn.sigmoid_cross_entropy_with_logits(logits=perturPredictions, labels=self.inputY)
                perturLosses = tf.nn.softmax_cross_entropy_with_logits(logits=perturPredictions, labels=self.inputY)
                perturLoss = tf.reduce_mean(perturLosses)

        self.loss = loss + perturLoss
        
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.AdamOptimizer(learning_rate)
        # 计算梯度,得到梯度和变量
        gradsAndVars = optimizer.compute_gradients(self.loss)
        # 将梯度应用到变量下，生成训练器
        self.trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)

        # 准确率
        correct_pred = tf.equal(tf.argmax(self.inputY, 1), self.y_pred_cls)
        self.acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        # self.loss = loss
        
        
    def _Bi_LSTMAttention(self, embeddedWords):
        # 定义两层双向LSTM的模型结构
        with tf.name_scope("Bi-LSTM"):
            fwHiddenLayers = []
            bwHiddenLayers = []
            for idx, hiddenSize in enumerate(hiddenSizes):
                with tf.name_scope("Bi-LSTM" + str(idx)):
                    # 定义前向网络结构
                    lstmFwCell = tf.nn.rnn_cell.DropoutWrapper(
                        tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                        output_keep_prob=self.dropoutKeepProb)

                    # 定义反向网络结构
                    lstmBwCell = tf.nn.rnn_cell.DropoutWrapper(
                        tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                        output_keep_prob=self.dropoutKeepProb)

                fwHiddenLayers.append(lstmFwCell)
                bwHiddenLayers.append(lstmBwCell)

            # 实现多层的LSTM结构， state_is_tuple=True，则状态会以元祖的形式组合(h, c)，否则列向拼接
            fwMultiLstm = tf.nn.rnn_cell.MultiRNNCell(cells=fwHiddenLayers, state_is_tuple=True)
            bwMultiLstm = tf.nn.rnn_cell.MultiRNNCell(cells=bwHiddenLayers, state_is_tuple=True)
            # 采用动态rnn，可以动态地输入序列的长度，若没有输入，则取序列的全长
            # outputs是一个元组(output_fw, output_bw), 其中两个元素的维度都是[batch_size, max_time, hidden_size], fw和bw的hiddensize一样
            # self.current_state是最终的状态，二元组(state_fw, state_bw), state_fw=[batch_size, s], s是一个元组(h, c)
            outputs, self.current_state = tf.nn.bidirectional_dynamic_rnn(fwMultiLstm, bwMultiLstm,
                                                                          self.embeddedWords, dtype=tf.float64,
                                                                          scope="bi-lstm" + str(idx))

        # 在bi-lstm+attention论文中，将前向和后向的输出相加
        with tf.name_scope("Attention"):
            H = outputs[0] + outputs[1]

            # 得到attention的输出
            output = self.attention(H)
            outputSize = hiddenSizes[-1]
            print("outputSize:{}".format(outputSize))

        # 全连接层的输出
        with tf.name_scope("output"):
            outputW = tf.get_variable(
                "outputW", dtype=tf.float64,
                shape=[outputSize, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())

            outputB = tf.Variable(tf.constant(0.1, dtype=tf.float64, shape=[num_classes]), name="outputB")

            predictions = tf.nn.xw_plus_b(output, outputW, outputB, name="predictions")

            return predictions

    def attention(self, H):
        """
        利用Attention机制得到句子的向量表示
        """
        # 获得最后一层lstm神经元的数量
        hiddenSize = hiddenSizes[-1]

        # 初始化一个权重向量，是可训练的参数
        W = tf.Variable(tf.random_normal([hiddenSize], stddev=0.1, dtype=tf.float64))

        # 对bi-lstm的输出用激活函数做非线性转换
        M = tf.tanh(H)

        # 对W和M做矩阵运算，W=[batch_size, time_step, hidden_size], 计算前做维度转换成[batch_size * time_step, hidden_size]
        # newM = [batch_size, time_step, 1], 每一个时间步的输出由向量转换成一个数字
        newM = tf.matmul(tf.reshape(M, [-1, hiddenSize]), tf.reshape(W, [-1, 1]))

        # 对newM做维度转换成[batch_size, time_step]
        restoreM = tf.reshape(newM, [-1, seq_length])

        # 用softmax做归一化处理[batch_size, time_step]
        self.alpha = tf.nn.softmax(restoreM)

        # 利用求得的alpha的值对H进行加权求和，用矩阵运算直接操作
        r = tf.matmul(tf.transpose(H, [0, 2, 1]), tf.reshape(self.alpha, [-1, seq_length, 1]))

        # 将三维压缩成二维sequeezeR = [batch_size, hissen_size]
        sequeezeR = tf.squeeze(r)

        sentenceRepren = tf.tanh(sequeezeR)

        # 对attention的输出可以做dropout处理
        output = tf.nn.dropout(sentenceRepren, self.dropoutKeepProb)

        return output

    def _normalize(self, wordEmbedding, weights):
        """
        对word embedding 结合权重做标准化处理
        """
        mean = tf.matmul(weights, wordEmbedding)
        powWordEmbedding = tf.pow(wordEmbedding - mean, 2.)

        var = tf.matmul(weights, powWordEmbedding)
        stddev = tf.sqrt(1e-6 + var)

        return (wordEmbedding - mean) / stddev

    def _addPerturbation(self, embedded, loss):
        """
        添加波动到word embedding
        """
        grad, = tf.gradients(
            loss,
            embedded,
            aggregation_method=tf.AggregationMethod.EXPERIMENTAL_ACCUMULATE_N)
        grad = tf.stop_gradient(grad)
        perturb = self._scaleL2(grad, epsilon)
        # print("perturbSize:{}".format(embedded+perturb))
        return embedded + perturb

    def _scaleL2(self, x, norm_length):
        # shape(x) = [batch, num_step, d]
        # divide x by max(abs(x)) for a numerically stable L2 norm
        # 2norm(x) = a * 2norm(x/a)
        # scale over the full sequence, dim(1, 2)
        alpha = tf.reduce_max(tf.abs(x), (1, 2), keep_dims=True) + 1e-12
        l2_norm = alpha * tf.sqrt(tf.reduce_sum(tf.pow(x / alpha, 2), (1, 2), keep_dims=True) + 1e-6)
        x_unit = x / l2_norm
        return norm_length * x_unit

In [8]:
hiddenSizes = [128]  # 定义LSTM的隐藏层（一层，128个神经元）
epsilon = 5

num_filters = 256
kernel_size = 5
hidden_dim = 128
learning_rate = 1e-3
dropout_keep_prob = 0.5

num_epochs = 50
batch_size = 64
print_per_batch = 30  # 每多少轮输出一次结果

lstm = AdversarailLSTM(embedding)

outputSize:128
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


perturbSize:Tensor("perturloss/Bi-LSTM/add_2:0", shape=(?, 41, 100), dtype=float64)
outputSize:128


In [10]:
kf = KFold(n_splits=10)
split_info = {
    # "random": False,
    "expert": [20, 4],
    "bundle": [920, 1],
    "table": [37, 3]
}
test_acc_split = []
for split_type,info in split_info.items():
    train_data,test_data = dataset_split(info)
    test_acc_split.append(train_split_data(lstm, train_data, test_data, split_type))

expert
Fold:  1
Training and evaluating...
Epoch: 1
Iter: 100, Train Loss:  3.129, Train Acc: 53.12%, Val Loss:  3.234, Val Acc: 50.09%, Time: 31.01s *
Iter: 200, Train Loss:  2.312, Train Acc: 59.38%, Val Loss:  2.217, Val Acc: 64.58%, Time: 61.12s *
Epoch: 2
Iter: 300, Train Loss:   1.22, Train Acc: 79.69%, Val Loss:  1.917, Val Acc: 70.84%, Time: 29.68s *
Iter: 400, Train Loss:  1.367, Train Acc: 76.56%, Val Loss:  1.744, Val Acc: 73.88%, Time: 59.87s *
Epoch: 3
Iter: 500, Train Loss: 0.8094, Train Acc: 92.19%, Val Loss:  1.696, Val Acc: 75.31%, Time: 29.08s *
Iter: 600, Train Loss: 0.3819, Train Acc: 96.88%, Val Loss:  1.618, Val Acc: 75.13%, Time: 59.39s 
Epoch: 4
Iter: 700, Train Loss: 0.9812, Train Acc: 87.50%, Val Loss:  1.595, Val Acc: 74.78%, Time: 28.63s 
Iter: 800, Train Loss: 0.7281, Train Acc: 89.06%, Val Loss:  1.679, Val Acc: 74.15%, Time: 58.99s 
Epoch: 5
Iter: 900, Train Loss: 0.6569, Train Acc: 85.94%, Val Loss:  1.783, Val Acc: 76.03%, Time: 27.93s *
Iter: 1000, Tra

Training and evaluating...
Epoch: 1
Iter: 100, Train Loss:  2.705, Train Acc: 67.19%, Val Loss:  2.966, Val Acc: 55.86%, Time: 32.02s *
Epoch: 2
Iter: 200, Train Loss:  1.448, Train Acc: 84.38%, Val Loss:  2.353, Val Acc: 61.99%, Time: 6.08s *
Iter: 300, Train Loss:  0.856, Train Acc: 89.06%, Val Loss:  2.052, Val Acc: 66.92%, Time: 37.33s *
Epoch: 3
Iter: 400, Train Loss: 0.9231, Train Acc: 87.50%, Val Loss:  2.174, Val Acc: 67.35%, Time: 9.79s *
Iter: 500, Train Loss:  0.776, Train Acc: 85.94%, Val Loss:  2.177, Val Acc: 68.17%, Time: 41.09s *
Epoch: 4
Iter: 600, Train Loss: 0.7682, Train Acc: 89.06%, Val Loss:  2.432, Val Acc: 66.87%, Time: 13.66s 
Iter: 700, Train Loss: 0.2672, Train Acc: 95.31%, Val Loss:  2.475, Val Acc: 68.21%, Time: 44.96s *
Epoch: 5
Iter: 800, Train Loss: 0.3571, Train Acc: 93.75%, Val Loss:   2.66, Val Acc: 67.40%, Time: 17.40s 
Iter: 900, Train Loss: 0.4479, Train Acc: 90.62%, Val Loss:  2.526, Val Acc: 69.84%, Time: 48.62s *
Epoch: 6
Iter: 1000, Train Loss:

Iter: 300, Train Loss:  1.548, Train Acc: 71.88%, Val Loss:  2.124, Val Acc: 65.36%, Time: 30.51s *
Iter: 400, Train Loss: 0.7129, Train Acc: 89.13%, Val Loss:  1.855, Val Acc: 69.75%, Time: 60.89s *
Epoch: 3
Iter: 500, Train Loss: 0.7279, Train Acc: 89.06%, Val Loss:  1.844, Val Acc: 70.95%, Time: 30.48s *
Iter: 600, Train Loss: 0.5064, Train Acc: 93.48%, Val Loss:  1.905, Val Acc: 71.35%, Time: 60.83s *
Epoch: 4
Iter: 700, Train Loss: 0.3503, Train Acc: 96.88%, Val Loss:  1.775, Val Acc: 72.47%, Time: 30.38s *
Iter: 800, Train Loss: 0.5179, Train Acc: 89.13%, Val Loss:  1.803, Val Acc: 72.23%, Time: 60.66s 
Epoch: 5
Iter: 900, Train Loss: 0.1472, Train Acc: 100.00%, Val Loss:  1.919, Val Acc: 71.99%, Time: 30.42s 
Iter: 1000, Train Loss: 0.2928, Train Acc: 97.83%, Val Loss:  1.708, Val Acc: 75.34%, Time: 60.77s *
Epoch: 6
Iter: 1100, Train Loss: 0.2558, Train Acc: 96.88%, Val Loss:  1.919, Val Acc: 71.75%, Time: 30.50s 
Iter: 1200, Train Loss:  0.228, Train Acc: 97.83%, Val Loss:  1.

Iter: 600, Train Loss: 0.8623, Train Acc: 86.84%, Val Loss:  1.716, Val Acc: 74.62%, Time: 60.97s *
Epoch: 4
Iter: 700, Train Loss:   0.48, Train Acc: 93.75%, Val Loss:  1.895, Val Acc: 73.83%, Time: 30.51s 
Iter: 800, Train Loss: 0.5561, Train Acc: 92.11%, Val Loss:  1.742, Val Acc: 75.26%, Time: 60.95s *
Epoch: 5
Iter: 900, Train Loss: 0.5725, Train Acc: 90.62%, Val Loss:  1.865, Val Acc: 74.94%, Time: 30.60s 
Iter: 1000, Train Loss: 0.3329, Train Acc: 94.74%, Val Loss:  1.869, Val Acc: 74.54%, Time: 61.22s 
Epoch: 6
Iter: 1100, Train Loss: 0.4644, Train Acc: 93.75%, Val Loss:  2.054, Val Acc: 74.15%, Time: 30.50s 
Iter: 1200, Train Loss: 0.2627, Train Acc: 97.37%, Val Loss:  2.014, Val Acc: 74.62%, Time: 61.15s 
Epoch: 7
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.56      0.82      0.67       114
                   Filter       0.54      0.74 

Training and evaluating...
Epoch: 1
Iter: 100, Train Loss:  3.155, Train Acc: 53.12%, Val Loss:  3.463, Val Acc: 44.86%, Time: 31.43s *
Epoch: 2
Iter: 200, Train Loss:  2.104, Train Acc: 65.62%, Val Loss:  2.568, Val Acc: 58.92%, Time: 2.14s *
Iter: 300, Train Loss:  1.127, Train Acc: 81.25%, Val Loss:  2.337, Val Acc: 62.77%, Time: 32.75s *
Epoch: 3
Iter: 400, Train Loss: 0.7135, Train Acc: 87.50%, Val Loss:  2.411, Val Acc: 64.98%, Time: 2.72s *
Iter: 500, Train Loss: 0.8636, Train Acc: 89.06%, Val Loss:  2.239, Val Acc: 66.33%, Time: 33.50s *
Epoch: 4
Iter: 600, Train Loss: 0.4336, Train Acc: 95.31%, Val Loss:  2.265, Val Acc: 68.62%, Time: 3.32s *
Iter: 700, Train Loss:  0.505, Train Acc: 92.19%, Val Loss:   2.23, Val Acc: 66.48%, Time: 34.08s 
Epoch: 5
Iter: 800, Train Loss: 0.4315, Train Acc: 93.75%, Val Loss:  2.656, Val Acc: 65.12%, Time: 3.90s 
Iter: 900, Train Loss:  0.254, Train Acc: 95.31%, Val Loss:  2.577, Val Acc: 66.41%, Time: 34.70s 
Epoch: 6
Iter: 1000, Train Loss: 0.

Training and evaluating...
Epoch: 1
Iter: 100, Train Loss:  3.049, Train Acc: 48.44%, Val Loss:  3.342, Val Acc: 44.82%, Time: 31.46s *
Epoch: 2
Iter: 200, Train Loss:  2.242, Train Acc: 64.06%, Val Loss:  2.481, Val Acc: 60.04%, Time: 2.17s *
Iter: 300, Train Loss:  1.273, Train Acc: 78.12%, Val Loss:  2.039, Val Acc: 66.60%, Time: 32.76s *
Epoch: 3
Iter: 400, Train Loss: 0.9421, Train Acc: 82.81%, Val Loss:  1.869, Val Acc: 68.50%, Time: 2.75s *
Iter: 500, Train Loss: 0.9964, Train Acc: 85.94%, Val Loss:  1.804, Val Acc: 72.59%, Time: 33.52s *
Epoch: 4
Iter: 600, Train Loss: 0.5853, Train Acc: 92.19%, Val Loss:  1.718, Val Acc: 75.05%, Time: 3.35s *
Iter: 700, Train Loss: 0.4241, Train Acc: 93.75%, Val Loss:  1.969, Val Acc: 72.73%, Time: 33.96s 
Epoch: 5
Iter: 800, Train Loss: 0.2677, Train Acc: 98.44%, Val Loss:  1.872, Val Acc: 72.94%, Time: 3.93s 
Iter: 900, Train Loss: 0.1798, Train Acc: 100.00%, Val Loss:  2.004, Val Acc: 72.80%, Time: 34.56s 
Epoch: 6
Iter: 1000, Train Loss: 0

Epoch: 2
Iter: 200, Train Loss:    2.0, Train Acc: 67.19%, Val Loss:   2.45, Val Acc: 59.37%, Time: 2.14s *
Iter: 300, Train Loss:  1.171, Train Acc: 81.25%, Val Loss:  2.033, Val Acc: 66.45%, Time: 32.69s *
Epoch: 3
Iter: 400, Train Loss: 0.5959, Train Acc: 90.62%, Val Loss:  2.031, Val Acc: 67.24%, Time: 2.71s *
Iter: 500, Train Loss: 0.6809, Train Acc: 95.31%, Val Loss:  1.923, Val Acc: 70.24%, Time: 33.37s *
Epoch: 4
Iter: 600, Train Loss: 0.4798, Train Acc: 93.75%, Val Loss:  1.908, Val Acc: 70.10%, Time: 3.28s 
Iter: 700, Train Loss: 0.3822, Train Acc: 93.75%, Val Loss:  2.007, Val Acc: 69.81%, Time: 33.99s 
Epoch: 5
Iter: 800, Train Loss: 0.3285, Train Acc: 96.88%, Val Loss:  1.971, Val Acc: 70.60%, Time: 3.89s *
Iter: 900, Train Loss: 0.3015, Train Acc: 93.75%, Val Loss:  2.178, Val Acc: 68.67%, Time: 34.66s 
Epoch: 6
Iter: 1000, Train Loss: 0.1721, Train Acc: 98.44%, Val Loss:  2.288, Val Acc: 67.95%, Time: 4.45s 
Iter: 1100, Train Loss: 0.1632, Train Acc: 100.00%, Val Loss:  

Epoch: 4
Iter: 600, Train Loss: 0.5416, Train Acc: 89.06%, Val Loss:  2.868, Val Acc: 60.19%, Time: 8.01s 
Iter: 700, Train Loss: 0.4096, Train Acc: 95.31%, Val Loss:  3.198, Val Acc: 59.01%, Time: 39.09s 
Epoch: 5
Iter: 800, Train Loss: 0.5091, Train Acc: 92.19%, Val Loss:  3.019, Val Acc: 61.90%, Time: 10.04s 
Iter: 900, Train Loss: 0.5786, Train Acc: 89.06%, Val Loss:  3.117, Val Acc: 61.25%, Time: 41.07s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.46      0.40      0.43       180
                   Filter       0.56      0.34      0.42       191
    Compute Derived Value       0.69      0.43      0.53       167
            Find Extremum       0.62      0.86      0.72       173
                     Sort       0.71      0.97      0.82       146
          Determine Range       0.76      0.61      0.68       183
Characterize Distribution       0

Epoch: 7
Iter: 1200, Train Loss: 0.3803, Train Acc: 95.31%, Val Loss:  2.506, Val Acc: 71.05%, Time: 19.65s 
Iter: 1300, Train Loss: 0.2449, Train Acc: 95.31%, Val Loss:  2.472, Val Acc: 71.21%, Time: 51.01s 
Epoch: 8
Iter: 1400, Train Loss: 0.2922, Train Acc: 95.31%, Val Loss:  2.545, Val Acc: 71.31%, Time: 22.59s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.59      0.67      0.63       174
                   Filter       0.64      0.72      0.68       152
    Compute Derived Value       0.64      0.55      0.59       228
            Find Extremum       0.87      0.72      0.79       218
                     Sort       0.88      0.85      0.86       171
          Determine Range       0.68      0.59      0.63       180
Characterize Distribution       0.75      0.79      0.77       199
           Find Anomalies       0.82      0.63      0.71     

Iter: 1100, Train Loss: 0.2051, Train Acc: 96.88%, Val Loss:  2.007, Val Acc: 74.02%, Time: 38.32s 
Epoch: 7
Iter: 1200, Train Loss: 0.1001, Train Acc: 98.44%, Val Loss:  2.174, Val Acc: 71.18%, Time: 8.76s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.66      0.78      0.71       146
                   Filter       0.57      0.64      0.60       178
    Compute Derived Value       0.49      0.75      0.60       145
            Find Extremum       0.85      0.56      0.68       172
                     Sort       0.76      0.73      0.75       142
          Determine Range       0.79      0.74      0.77       154
Characterize Distribution       0.87      0.79      0.83       155
           Find Anomalies       0.76      0.70      0.73       148
                  Cluster       0.79      0.78      0.78       156
                Correlate       0.85 

In [9]:
kf = KFold(n_splits=10)
test_acc_split = []
for split_type,info in split_info.items():
    train_data,test_data = dataset_split(info)
    test_acc_split.append(train_split_data(lstm, train_data, test_data, split_type))

random
Fold:  1
Training and evaluating...
Epoch: 1
Iter: 30, Train Loss:  4.414, Train Acc: 23.44%, Val Loss:  4.496, Val Acc: 20.73%, Time: 8.94s *
Iter: 60, Train Loss:  4.081, Train Acc: 37.50%, Val Loss:  4.005, Val Acc: 33.90%, Time: 14.99s *
Iter: 90, Train Loss:   3.11, Train Acc: 48.44%, Val Loss:  3.324, Val Acc: 45.30%, Time: 21.09s *
Iter: 120, Train Loss:  2.557, Train Acc: 64.06%, Val Loss:  2.783, Val Acc: 57.26%, Time: 27.10s *
Iter: 150, Train Loss:  2.509, Train Acc: 59.38%, Val Loss:  2.489, Val Acc: 60.40%, Time: 33.15s *
Iter: 180, Train Loss:  2.189, Train Acc: 70.31%, Val Loss:  2.067, Val Acc: 67.81%, Time: 39.19s *
Epoch: 2
Iter: 210, Train Loss:  1.462, Train Acc: 79.69%, Val Loss:  1.929, Val Acc: 70.58%, Time: 3.33s *
Iter: 240, Train Loss:   1.39, Train Acc: 76.56%, Val Loss:  1.716, Val Acc: 72.86%, Time: 9.38s *
Iter: 270, Train Loss:  1.408, Train Acc: 79.69%, Val Loss:  1.533, Val Acc: 76.14%, Time: 15.43s *
Iter: 300, Train Loss: 0.8514, Train Acc: 87.

Iter: 2460, Train Loss: 0.1755, Train Acc: 95.31%, Val Loss: 0.7304, Val Acc: 91.45%, Time: 17.20s 
Iter: 2490, Train Loss: 0.0112, Train Acc: 100.00%, Val Loss: 0.7191, Val Acc: 91.31%, Time: 23.27s 
Iter: 2520, Train Loss: 0.05898, Train Acc: 98.44%, Val Loss: 0.7248, Val Acc: 92.02%, Time: 29.37s 
Iter: 2550, Train Loss: 0.05714, Train Acc: 98.44%, Val Loss: 0.7664, Val Acc: 91.88%, Time: 35.39s 
Epoch: 14
Iter: 2580, Train Loss: 0.01778, Train Acc: 100.00%, Val Loss: 0.7104, Val Acc: 91.74%, Time: 2.54s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.87      0.87      0.87       143
                   Filter       0.92      0.89      0.90       170
    Compute Derived Value       0.87      0.96      0.91       138
            Find Extremum       0.98      0.98      0.98       180
                     Sort       0.96      0.98      0.97       110

Iter: 1860, Train Loss: 0.03261, Train Acc: 100.00%, Val Loss: 0.8579, Val Acc: 88.96%, Time: 16.24s 
Iter: 1890, Train Loss: 0.06901, Train Acc: 100.00%, Val Loss:  0.834, Val Acc: 89.74%, Time: 22.32s 
Iter: 1920, Train Loss: 0.02308, Train Acc: 100.00%, Val Loss: 0.8916, Val Acc: 88.39%, Time: 28.41s 
Iter: 1950, Train Loss: 0.02041, Train Acc: 100.00%, Val Loss: 0.8813, Val Acc: 88.53%, Time: 34.55s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.90      0.89      0.89       148
                   Filter       0.85      0.85      0.85       163
    Compute Derived Value       0.85      0.91      0.88       153
            Find Extremum       0.91      0.93      0.92       160
                     Sort       0.94      0.95      0.94       124
          Determine Range       0.83      0.83      0.83       120
Characterize Distribution       0.97  

Iter: 1890, Train Loss: 0.2174, Train Acc: 96.88%, Val Loss: 0.8247, Val Acc: 90.17%, Time: 22.26s 
Iter: 1920, Train Loss: 0.1377, Train Acc: 96.88%, Val Loss: 0.8505, Val Acc: 88.53%, Time: 28.34s 
Iter: 1950, Train Loss: 0.1091, Train Acc: 98.44%, Val Loss: 0.7132, Val Acc: 90.60%, Time: 34.47s 
Iter: 1980, Train Loss: 0.0194, Train Acc: 100.00%, Val Loss:   0.82, Val Acc: 89.39%, Time: 40.49s 
Epoch: 11
Iter: 2010, Train Loss: 0.3003, Train Acc: 95.31%, Val Loss:  0.781, Val Acc: 90.17%, Time: 6.04s 
Iter: 2040, Train Loss: 0.1353, Train Acc: 98.44%, Val Loss: 0.8405, Val Acc: 90.03%, Time: 12.07s 
Iter: 2070, Train Loss: 0.1537, Train Acc: 96.88%, Val Loss: 0.8023, Val Acc: 90.60%, Time: 18.08s 
Iter: 2100, Train Loss: 0.06733, Train Acc: 100.00%, Val Loss:  0.894, Val Acc: 89.74%, Time: 24.08s 
Iter: 2130, Train Loss: 0.05315, Train Acc: 100.00%, Val Loss: 0.8548, Val Acc: 89.53%, Time: 30.06s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
  

Iter: 1740, Train Loss: 0.07308, Train Acc: 100.00%, Val Loss: 0.9676, Val Acc: 87.54%, Time: 32.59s 
Iter: 1770, Train Loss: 0.1315, Train Acc: 96.88%, Val Loss:  1.025, Val Acc: 86.32%, Time: 38.58s 
Epoch: 10
Iter: 1800, Train Loss: 0.06518, Train Acc: 100.00%, Val Loss: 0.9404, Val Acc: 88.39%, Time: 4.19s *
Iter: 1830, Train Loss: 0.05791, Train Acc: 100.00%, Val Loss: 0.9107, Val Acc: 88.82%, Time: 10.22s *
Iter: 1860, Train Loss: 0.02594, Train Acc: 100.00%, Val Loss:  1.148, Val Acc: 87.32%, Time: 16.15s 
Iter: 1890, Train Loss: 0.05869, Train Acc: 100.00%, Val Loss: 0.9907, Val Acc: 87.68%, Time: 22.12s 
Iter: 1920, Train Loss: 0.1431, Train Acc: 95.31%, Val Loss:  1.009, Val Acc: 87.89%, Time: 28.18s 
Iter: 1950, Train Loss: 0.07602, Train Acc: 100.00%, Val Loss:  1.014, Val Acc: 87.25%, Time: 34.16s 
Iter: 1980, Train Loss: 0.02152, Train Acc: 100.00%, Val Loss:  1.033, Val Acc: 86.82%, Time: 40.22s 
Epoch: 11
Iter: 2010, Train Loss: 0.03747, Train Acc: 100.00%, Val Loss:  1

...

Iter: 900, Train Loss: 0.4939, Train Acc: 95.31%, Val Loss:  1.691, Val Acc: 75.95%, Time: 20.35s 
Iter: 930, Train Loss: 0.3376, Train Acc: 93.75%, Val Loss:  1.566, Val Acc: 76.19%, Time: 26.18s 
Iter: 960, Train Loss: 0.2566, Train Acc: 98.44%, Val Loss:  1.521, Val Acc: 77.72%, Time: 32.02s *
Iter: 990, Train Loss: 0.2578, Train Acc: 95.31%, Val Loss:  1.537, Val Acc: 77.89%, Time: 37.75s *
Epoch: 6
Iter: 1020, Train Loss: 0.3832, Train Acc: 92.19%, Val Loss:  1.712, Val Acc: 75.54%, Time: 4.32s 
Iter: 1050, Train Loss: 0.2281, Train Acc: 95.31%, Val Loss:  1.672, Val Acc: 76.92%, Time: 10.19s 
Iter: 1080, Train Loss: 0.3194, Train Acc: 96.88%, Val Loss:   1.84, Val Acc: 75.54%, Time: 16.01s 
Iter: 1110, Train Loss: 0.2017, Train Acc: 98.44%, Val Loss:   1.63, Val Acc: 76.84%, Time: 21.81s 
Iter: 1140, Train Loss: 0.2653, Train Acc: 95.31%, Val Loss:  1.543, Val Acc: 77.08%, Time: 27.60s 
Iter: 1170, Train Loss: 0.2229, Train Acc: 96.88%, Val Loss:  1.612, Val Acc: 78.45%, Time: 33

Epoch: 7
Iter: 1170, Train Loss: 0.08347, Train Acc: 98.44%, Val Loss:  2.857, Val Acc: 69.33%, Time: 2.63s 
Iter: 1200, Train Loss: 0.2152, Train Acc: 96.88%, Val Loss:  3.109, Val Acc: 68.37%, Time: 8.85s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.60      0.64      0.62       112
                   Filter       0.68      0.61      0.65       127
    Compute Derived Value       0.80      0.55      0.65       208
            Find Extremum       0.78      0.83      0.80       329
                     Sort       0.84      0.72      0.78       120
          Determine Range       0.44      0.47      0.46       118
Characterize Distribution       0.61      0.69      0.65       180
           Find Anomalies       0.64      0.63      0.64       145
                  Cluster       0.71      0.66      0.68       141
                Correlate       0.66 

Iter: 120, Train Loss:  2.626, Train Acc: 60.94%, Val Loss:  2.885, Val Acc: 55.71%, Time: 26.75s *
Iter: 150, Train Loss:  1.953, Train Acc: 73.44%, Val Loss:  2.544, Val Acc: 59.08%, Time: 33.15s *
Iter: 180, Train Loss:   2.62, Train Acc: 56.25%, Val Loss:   2.34, Val Acc: 63.18%, Time: 39.48s *
Epoch: 2
Iter: 210, Train Loss:   1.34, Train Acc: 78.12%, Val Loss:  2.088, Val Acc: 66.72%, Time: 4.58s *
Iter: 240, Train Loss:  1.504, Train Acc: 75.00%, Val Loss:    2.0, Val Acc: 67.75%, Time: 10.88s *
Iter: 270, Train Loss:  1.229, Train Acc: 78.12%, Val Loss:  2.039, Val Acc: 68.38%, Time: 17.16s *
Iter: 300, Train Loss:  0.831, Train Acc: 87.50%, Val Loss:  2.034, Val Acc: 68.66%, Time: 23.57s *
Iter: 330, Train Loss:  1.281, Train Acc: 78.12%, Val Loss:  1.949, Val Acc: 70.26%, Time: 29.92s *
Iter: 360, Train Loss: 0.9772, Train Acc: 85.94%, Val Loss:   1.86, Val Acc: 69.98%, Time: 36.31s 
Epoch: 3
Iter: 390, Train Loss: 0.9694, Train Acc: 85.94%, Val Loss:  1.851, Val Acc: 71.06%,

Iter: 1110, Train Loss: 0.3052, Train Acc: 96.88%, Val Loss:   3.01, Val Acc: 62.29%, Time: 19.17s 
Iter: 1140, Train Loss: 0.07053, Train Acc: 98.44%, Val Loss:  3.415, Val Acc: 59.91%, Time: 24.80s 
Iter: 1170, Train Loss: 0.4058, Train Acc: 95.31%, Val Loss:  2.992, Val Acc: 62.66%, Time: 30.44s 
Iter: 1200, Train Loss: 0.3213, Train Acc: 96.88%, Val Loss:   3.09, Val Acc: 60.46%, Time: 36.16s 
Epoch: 7
Iter: 1230, Train Loss: 0.3383, Train Acc: 93.75%, Val Loss:   3.15, Val Acc: 61.47%, Time: 3.06s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.52      0.47      0.49       118
                   Filter       0.51      0.36      0.43       129
    Compute Derived Value       0.58      0.69      0.63       124
            Find Extremum       0.50      0.79      0.61       114
                     Sort       0.89      0.93      0.91       120
    

InvalidArgumentError: In[0] is not a matrix
	 [[Node: loss/Bi-LSTM/output/predictions/MatMul = MatMul[T=DT_DOUBLE, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](loss/Bi-LSTM/Attention/dropout/mul, Bi-LSTM/outputW/read)]]
	 [[Node: Adam/update/_46 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_3036_Adam/update", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'loss/Bi-LSTM/output/predictions/MatMul', defined at:
  File "/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/site-pack